In [ ]:
!python3 -m venv env
!source env/bin/activate
!pip install sentencepiece
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets matplotlib einops
!pip install accelerate
!pip install -q wandb -U


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!source env/bin/activate
!export CUDA_VISIBLE_DEVICES=0,1

: 

In [34]:
from datasets import load_dataset
from dataset import PlatypusDataset, track_performance
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
from peft import prepare_model_for_kbit_training
from accelerate import Accelerator
from peft import LoraConfig, get_peft_model
import wandb, os
import transformers
from datetime import datetime
from accelerate import PartialState
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use only one GPU

accelerator = Accelerator()

In [35]:
def formatting_func(example):
    text = f"### Question: {example['instruction']}\n ### Answer: {example['output']}"
    return text

In [36]:

@track_performance
def finetune_new_model_on_dataset(train_dataset, val_dataset, base_model_id, base_model, source_name):    
    model = prepare_model_for_kbit_training(base_model)
    model = get_peft_model(model, config)
    # print_trainable_parameters(model)
    
    # Apply the accelerator. You can comment this out to remove the accelerator.
    model = accelerator.prepare_model(model)

    if torch.cuda.device_count() > 1: # If more than 1 GPU
        model.is_parallelizable = True
        model.model_parallel = True

    project = "llama-finetune/ablation/"
    base_model_name = f"meta-llama/{base_model_id}"
    run_name = base_model_name + "-" + project + source_name
    output_dir = "./" + run_name
    
    tokenizer.pad_token = tokenizer.eos_token

    trainer = transformers.Trainer(
        model=model,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_val_dataset,
        args=transformers.TrainingArguments(
            output_dir=output_dir,
            warmup_steps=5,
            per_device_train_batch_size=2,
            gradient_checkpointing=True,
            gradient_accumulation_steps=4,
            max_steps=1000,
            learning_rate=2.5e-5,
            logging_steps=50,
            bf16=True,
            optim="paged_adamw_8bit",
            logging_dir="./logs",        # Directory for storing logs
            save_strategy="steps",       # Save the model checkpoint every logging step
            save_steps=25,                # Save checkpoints every 50 steps
            evaluation_strategy="steps", # Evaluate the model every logging step
            eval_steps=50,               # Evaluate and save checkpoints every 50 steps
            do_eval=True,                # Perform evaluation at the end of training
            report_to="wandb",           # Comment this out if you don't want to use weights & baises
            run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M'),}",
            local_rank=-1,
            ddp_find_unused_parameters=False,
            no_cuda=False,
            dataloader_pin_memory=False
        ),
        data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    )
    
    model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
    trainer.train()

    # base_model = AutoModelForCausalLM.from_pretrained(
    #     base_model_id,
    #     quantization_config=bnb_config)
    
    # eval_tokenizer = AutoTokenizer.from_pretrained(
    #     base_model_id,
    #     add_bos_token=True,
    #     trust_remote_code=True,
    # )


In [37]:
import matplotlib.pyplot as plt

def plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset):
    lengths = [len(x['input_ids']) for x in tokenized_train_dataset]
    lengths += [len(x['input_ids']) for x in tokenized_val_dataset]
    print(len(lengths))

    # Plotting the histogram
    plt.figure(figsize=(10, 6))
    plt.hist(lengths, bins=20, alpha=0.7, color='blue')
    plt.xlabel('Length of input_ids')
    plt.ylabel('Frequency')
    plt.title('Distribution of Lengths of input_ids')
    plt.show()

# plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset)

In [ ]:
wandb.login()
wandb_project = "llama-finetune"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project
    
config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

max_length = 2048 # This was an appropriate max length for my dataset

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

def generate_and_tokenize_prompt2(prompt):
    result = tokenizer(
        formatting_func(prompt),
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result
    
# base_model_id = "meta-llama/Llama-3.1-8B"
base_model_id = "HuggingFaceH4/tiny-random-LlamaForCausalLM"
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map={"": "cuda:0"},  # Force everything to cuda:0
    torch_dtype=torch.float16 )
model.gradient_checkpointing_enable()

tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
    use_fast=False, # needed for now, should be fixed soon
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token

accelerator = Accelerator()

dataset = PlatypusDataset()
data_sources = ['leetcode_ne', 'ARB', 'tigerbot-kaggle', 'MATH/PRM-800K', 'theoremqa', 'scienceqa', 'reclor', 'guanaco', 'scibench', 'airoboros']
for source in data_sources:
    print(f"source: {source}")
    train_dataset = dataset.get_train_data_without_source(source)
    val_dataset = dataset.get_val_data_without_source(source)
    # Modify data preparation
    tokenized_train_dataset = [
        {k: v.to(accelerator.device) for k, v in generate_and_tokenize_prompt2(point).items()}
        for point in train_dataset
    ]
    tokenized_val_dataset = [
        {k: v.to(accelerator.device) for k, v in generate_and_tokenize_prompt2(point).items()}
        for point in val_dataset
    ]
    print(f"Training set size: {len(train_dataset)}")
    print(f"Validation set size: {len(val_dataset)}")
    # plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset)
    finetune_new_model_on_dataset(train_dataset, val_dataset, base_model_id, model, accelerator, source)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `pad_token_id` should be positive but got -1. This will cause errors when batch generating, if there is padding. Please set `pad_token_id` explicitly as `model.generation_config.pad_token_id=PAD_TOKEN_ID` to avoid errors in generation
  warnings.warn(


In [ ]:
# train_val_split = ds.train_test_split(test_size=0.2)

# Extract the training and validation datasets
# train_dataset = train_val_split['train']
# val_dataset = train_val_split['test']

# Print the size of the datasets
# print(f"Training set size: {len(train_dataset)}")
# print(f"Validation set size: {len(val_dataset)}")

In [ ]:
# print(formatting_func(ds[0]))

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, DataCollatorForLanguageModeling
# from peft import LoraConfig, get_peft_model

In [ ]:
# base_model_id = "meta-llama/Llama-3.1-8B"
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(
#     base_model_id,
#     padding_side="left",
#     add_eos_token=True,
#     add_bos_token=True,
#     use_fast=False, # needed for now, should be fixed soon
# )
# tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# max_length = 2048 # This was an appropriate max length for my dataset

# def generate_and_tokenize_prompt2(prompt):
#     result = tokenizer(
#         formatting_func(prompt),
#         truncation=True,
#         max_length=max_length,
#         padding="max_length",
#     )
#     result["labels"] = result["input_ids"].copy()
#     return result

In [ ]:
# tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt2)
# tokenized_val_dataset = val_dataset.map(generate_and_tokenize_prompt2)

In [ ]:
# from peft import prepare_model_for_kbit_training

# model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
# print(model)

In [ ]:
# !pip install accelerate

In [ ]:
# from accelerate import Accelerator

# accelerator = Accelerator()

In [ ]:
# from peft import LoraConfig, get_peft_model

# config = LoraConfig(
#     r=8,
#     lora_alpha=16,
#     target_modules=[
#         "q_proj",
#         "k_proj",
#         "v_proj",
#         "o_proj",
#         "gate_proj",
#         "up_proj",
#         "down_proj",
#         "lm_head",
#     ],
#     bias="none",
#     lora_dropout=0.05,  # Conventional
#     task_type="CAUSAL_LM",
# )

# model = get_peft_model(model, config)
# print_trainable_parameters(model)

# # Apply the accelerator. You can comment this out to remove the accelerator.
# model = accelerator.prepare_model(model)

In [ ]:
# print(model)

In [ ]:
# !pip install -q wandb -U

# import wandb, os
# wandb.login()

# wandb_project = "llama-finetune"
# if len(wandb_project) > 0:
#     os.environ["WANDB_PROJECT"] = wandb_project

In [ ]:
# if torch.cuda.device_count() > 1: # If more than 1 GPU
#     model.is_parallelizable = True
#     model.model_parallel = True

In [ ]:
# import transformers
# from datetime import datetime

# sources = ["a", "b", "c"]
    
# project = "llama-finetune"
# base_model_name = "meta-llama/ablation/Llama-3.1-8B"
# run_name = base_model_name + "-" + project + source
# output_dir = "./" + run_name

# tokenizer.pad_token = tokenizer.eos_token

# trainer = transformers.Trainer(
#     model=model,
#     train_dataset=tokenized_train_dataset,
#     eval_dataset=tokenized_val_dataset,
#     args=transformers.TrainingArguments(
#         output_dir=output_dir,
#         warmup_steps=5,
#         per_device_train_batch_size=2,
#         gradient_checkpointing=True,
#         gradient_accumulation_steps=4,
#         max_steps=1000,
#         learning_rate=2.5e-5,
#         logging_steps=50,
#         bf16=True,
#         optim="paged_adamw_8bit",
#         logging_dir="./logs",        # Directory for storing logs
#         save_strategy="steps",       # Save the model checkpoint every logging step
#         save_steps=25,                # Save checkpoints every 50 steps
#         evaluation_strategy="steps", # Evaluate the model every logging step
#         eval_steps=50,               # Evaluate and save checkpoints every 50 steps
#         do_eval=True,                # Perform evaluation at the end of training
#         report_to="wandb",           # Comment this out if you don't want to use weights & baises
#         run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
#     ),
#     data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
# )

# model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
# trainer.train()

Reconnect the kernel after obtaining a good checkpoint

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# base_model_id = "meta-llama/Llama-3.1-8B"
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# base_model = AutoModelForCausalLM.from_pretrained(
#    base_model_id,
#    quantization_config=bnb_config,  # Same quantization config as before
#    device_map="auto",
#    trust_remote_code=True,
#)

# base_model = AutoModelForCausalLM.from_pretrained(
#     base_model_id,
#     quantization_config=bnb_config)

# eval_tokenizer = AutoTokenizer.from_pretrained(
#     base_model_id,
#     add_bos_token=True,
#     trust_remote_code=True,
# )

In [ ]:
# from peft import PeftModel

# ft_model = PeftModel.from_pretrained(base_model, "meta-llama/Llama-3.1-8B-llama-finetune/checkpoint-675")

In [ ]:
# from deepeval.models.base_model import DeepEvalBaseLLM
# from deepeval.benchmarks import HellaSwag
# from deepeval.benchmarks.tasks import HellaSwagTask
# import re

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B")

In [ ]:
# class Llama3(DeepEvalBaseLLM):
#     def __init__(
#         self,
#         model,
#         tokenizer
#     ):
#         self.model = model
#         self.tokenizer = tokenizer

#     def load_model(self):
#         return self.model

#     def generate(self, prompt: str) -> str:
#         sections = [section.strip() for section in prompt.split("\n\n") if section.strip()]

#         # Take the last section, including "Answer:" for context
#         prompt = sections[-2]

#         model = self.load_model()

#         device = "cuda" # the device to load the model onto

#         model_inputs = self.tokenizer([prompt], return_tensors="pt").to(device)
#         model.to(device)

#         generated_ids = model.generate(
#             **model_inputs, 
#             max_new_tokens=100, 
#             use_cache=True)
        
#         ans = self.tokenizer.batch_decode(generated_ids)[0]

#         match = re.search(r"Answer:\s*([A-D])", ans)

#         if match:
#             answer = match.group(1)
#         else:
#             answer = 'N/A'

#         return answer

#     async def a_generate(self, prompt: str) -> str:
#         return self.generate(prompt)

#     # This is optional.
#     def batch_generate(self, promtps: list[str]) -> list[str]:
#         model = self.load_model()
#         device = "cuda" # the device to load the model onto

#         new_p = []
#         for p in promtps:
#             sections = [section.strip() for section in prompt.split("\n\n") if section.strip()]
#             new_p.append(sections[-2])
            
#         model_inputs = self.tokenizer(
#             new_p,
#             padding=True,    # Ensure equal-length inputs
#             truncation=True, # Truncate inputs that exceed max_length
#             max_length=512,
#             return_tensors="pt").to(device)
#         model.to(device)

#         generated_ids = model.generate(
#             **model_inputs,
#             max_new_tokens=100, use_cache=True)
#         decoded_responses = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

#         res = []
#         for ans in decoded_responses:
#             match = re.search(r"Answer:\s*([A-D])", ans)
    
#             if match:
#                 res.append(match.group(1))
#             else:
#                 res.append('N/A')
#         return res

#     def get_model_name(self):
#         return "Llama 3"


# llama3 = Llama3(model=ft_model, tokenizer=tokenizer)

In [ ]:
# benchmark = HellaSwag(
#     tasks=[HellaSwagTask.APPLYING_SUNSCREEN, HellaSwagTask.SKATEBOARDING],
#     n_shots=3)

In [ ]:
# results = benchmark.evaluate(model=llama3, batch_size = 5)
# print("Task-specific Scoress: ", benchmark.task_scores)
# print("Detailed Predictions: ", benchmark.predictions)
# print(benchmark.overall_score)